1. [USDA CropScape data](https://nassgeodata.gmu.edu/CropScape/)
2. Select county AOI
3. Download county for 2017, **disable** frequency and mask, set projection to UTM 15
4. Extract and rename to `CDL_2017_<FIPS>.tif`, where `<FIPS>` is the county FIPS code (`200001` for Allen)

In [1]:
%%capture
%%script false  # disable to run cell

# Install required libraries
!pip install rasterio numpy geopandas rasterstats

In [2]:
# Load required libraries
import csv
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import rasterstats
import numpy as np
import geopandas as gpd

In [3]:
# Function to mask any raster by any value
def mask_raster(src_path, value_to_keep, dst_path):
    '''Entire crop raster -> Masked raster'''
    with rasterio.open(src_path) as src:
        raster_data = src.read()
        mask = np.isin(raster_data, value_to_keep, invert=True)
        masked_data = np.ma.masked_array(raster_data, mask=mask)
        meta = src.meta
    
    with rasterio.open(dst_path, 'w+', **meta) as dst:
        dst.write(masked_data)

In [4]:
# Function to convert crop raster to shp
def crop_to_shp(src_path, fips, crop):
    '''Masked crop raster -> Shapefile'''
    with rasterio.open(src_path) as src:
        raster_data = src.read(1)
        shapes_ = list(shapes(raster_data, mask=None, transform=src.transform))
    
    polygons = []
    for geom, val in shapes_:
        if val == crop:  # select only crop polygons
            polygons.append({'geometry': shape(geom)})
    
    gdf = gpd.GeoDataFrame(polygons)
    gdf = gdf.set_crs(src.crs)  # set CRS
    
    gdf['area'] = gdf.geometry.area
    
    output_shapefile = f'crop_{crop}_{fips}.shp'
    gdf.to_file(output_shapefile)

### TODO: Rename all county TOFI files to `TOFI_2017_<FIPS>.tif`

In [5]:
fipses = [20001]
# crops = [1, 5, 6, 23, 24, 26, 225, 226, 236, 237, 238, 240, 241, 254]
crops = [1, 5]

data = []

for crop in crops:
    # TODO: extract crop raster from state file
    try:
        for fips in fipses:
            crop_file = f'CDL_2017_{fips}.tif'
            tree_file = f'TOFI_2017_{fips}.tif'
            crop_mask_file = f'masked_{crop}_{crop_file}'
            tree_mask_file = f'masked_TOFI_2017_{fips}.tif'
            shp_file = f'crop_{crop}_{fips}.shp'

            mask_raster(crop_file, crop, crop_mask_file)  # TODO: update and move to memory when looping crops
            mask_raster(tree_file, 1, tree_mask_file)

            crop_to_shp(crop_mask_file, fips, crop)

            # TODO: Buffer

            # Count tree pixels
            # TODO: there must be a faster method than zonal_stats
            stats = rasterstats.zonal_stats(vectors=shp_file,
                                            raster=tree_mask_file,
                                            stats=['sum'],
                                            nodata=-9999,
                                            all_touched=True)

            tree_px = sum([x['sum'] for x in stats])

            # calc crop area
            # crop resolution is 30m, so 'area' is in m^2
            # tree resolution is 2m

            gdf = gpd.read_file(shp_file)
            crop_area = gdf['area'].sum()

            # pdf.plot()

            # assemble data output
            data.append({'fips': fips, 'crop_id': crop, 'crop_area_m2': crop_area, 'tree_area_m2': tree_px*4})
        
        except AttributeError: pass  # crop type not in county

# write data to csv
header = ['fips', 'crop_id', 'crop_area_m2', 'tree_area_m2']
with open('output.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(data)

AttributeError: You are calling a geospatial method on the GeoDataFrame, but the active geometry column to use has not been set. 
There are no existing columns with geometry data type. You can add a geometry column as the active geometry column with df.set_geometry. 